In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# ==============================
# 1. 设置JAG期刊图表样式
# ==============================

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
mpl.rcParams['mathtext.fontset'] = 'stix'

fig_width_cm = 17
fig_height_cm = 10
fig_width_inch = fig_width_cm / 2.54
fig_height_inch = fig_height_cm / 2.54

mpl.rcParams['axes.linewidth'] = 1.0
mpl.rcParams['lines.linewidth'] = 2.0
mpl.rcParams['lines.markersize'] = 6
mpl.rcParams['axes.labelsize'] = 11
mpl.rcParams['axes.titlesize'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 9
mpl.rcParams['figure.dpi'] = 300

# ==============================
# 2. 读取和处理数据
# ==============================

print("Loading data...")
df = pd.read_csv('/content/drive/MyDrive/results_base_rate_by_ecoregion.csv')
print(f"Data shape: {df.shape}")
print(f"Number of ecoregions: {df['ecoregion'].nunique()}")
print(f"Base rate range: {df['pfcl_base_rate_percent'].min()}% - {df['pfcl_base_rate_percent'].max()}%")

# 计算UA_adj统计量
print("\nCalculating UA_adj statistics...")
ua_adj_stats = df.groupby('pfcl_base_rate_percent')['UA_adj'].agg([
    'mean', 'std',
    lambda x: np.percentile(x, 25),
    lambda x: np.percentile(x, 75)
]).rename(columns={'<lambda_0>': 'q25', '<lambda_1>': 'q75'})

# ==============================
# 3. 定义原始数据
# ==============================

# 第一组: 未经调整的UA_original (平衡抽样下的乐观估计)
ua_orig_naive = np.array([
    0.922, 0.900, 0.923, 0.880, 0.903, 0.921, 0.969, 0.957, 0.988, 0.951,
    0.946, 0.963, 0.964, 0.969, 0.969, 0.958, 0.972, 0.933, 0.972, 0.962,
    0.953, 0.917, 0.954, 0.977, 0.929, 0.918, 0.912, 0.943, 0.909, 0.963,
    0.908, 0.902, 0.899, 0.970, 0.971
])

# 第二组: 经过空间LOBO计算的UA_original (空间校正后的稳健估计)
ua_orig_lobo = np.array([
    0.756, 0.743, 0.774, 0.865, 0.797, 0.662, 0.610, 0.808, 0.697, 0.603,
    0.962, 0.798, 0.876, 0.947, 0.773, 0.777, 0.963, 0.961, 0.971, 0.901,
    0.879, 0.897, 0.949, 0.921, 0.918, 0.929, 0.787, 0.903, 0.854, 0.857,
    0.597, 0.775, 0.743, 0.736, 0.805
])

# 假设: 我们可以为每个生态区计算空间LOBO调整因子
# 调整因子 = UA_orig_lobo / UA_orig_naive
spatial_adjustment_factors = ua_orig_lobo / ua_orig_naive

print(f"\nSpatial LOBO adjustment factors:")
print(f"  Mean factor: {np.mean(spatial_adjustment_factors):.4f}")
print(f"  Range: [{np.min(spatial_adjustment_factors):.4f}, {np.max(spatial_adjustment_factors):.4f}]")
print(f"  This represents average reduction of {(1-np.mean(spatial_adjustment_factors))*100:.1f}%")

# ==============================
# 4. 计算UA_adj (spatial LOBO adjusted)
# ==============================

print("\nCalculating UA_adj (spatial LOBO adjusted)...")

# 首先，我们需要为每个生态区应用空间调整因子
# 假设每个生态区的调整因子是恒定的，不随基础率变化

# 获取排序后的生态区列表
ecoregions_sorted = sorted(df['ecoregion'].unique())

# 创建生态区到调整因子的映射
# 注意：这里假设数组顺序与排序后的生态区顺序一致
if len(ecoregions_sorted) == len(spatial_adjustment_factors):
    eco_to_factor = {ecoregions_sorted[i]: spatial_adjustment_factors[i]
                     for i in range(len(ecoregions_sorted))}

    # 为每个数据点应用调整因子
    df['UA_adj_spatial'] = df.apply(
        lambda row: row['UA_adj'] * eco_to_factor.get(row['ecoregion'], 1.0),
        axis=1
    )

    # 计算调整后的统计量
    ua_adj_spatial_stats = df.groupby('pfcl_base_rate_percent')['UA_adj_spatial'].agg([
        'mean', 'std',
        lambda x: np.percentile(x, 25),
        lambda x: np.percentile(x, 75)
    ]).rename(columns={'<lambda_0>': 'q25', '<lambda_1>': 'q75'})

    print(f"  UA_adj_spatial mean: {df['UA_adj_spatial'].mean():.4f}")
    print(f"  UA_adj mean (before spatial adjustment): {df['UA_adj'].mean():.4f}")
    print(f"  Average reduction due to spatial adjustment: "
          f"{(1 - df['UA_adj_spatial'].mean()/df['UA_adj'].mean())*100:.1f}%")
else:
    print(f"  Warning: Number of ecoregions ({len(ecoregions_sorted)}) "
          f"does not match adjustment factors ({len(spatial_adjustment_factors)})")
    print(f"  Using average adjustment factor for all ecoregions")

    # 使用平均调整因子
    avg_factor = np.mean(spatial_adjustment_factors)
    df['UA_adj_spatial'] = df['UA_adj'] * avg_factor

    ua_adj_spatial_stats = df.groupby('pfcl_base_rate_percent')['UA_adj_spatial'].agg([
        'mean', 'std',
        lambda x: np.percentile(x, 25),
        lambda x: np.percentile(x, 75)
    ]).rename(columns={'<lambda_0>': 'q25', '<lambda_1>': 'q75'})

# 计算统计量函数
def calculate_stats(values, name):
    return {
        'name': name,
        'mean': np.mean(values),
        'median': np.median(values),
        'std': np.std(values),
        'q25': np.percentile(values, 25),
        'q75': np.percentile(values, 75),
        'min': np.min(values),
        'max': np.max(values),
        'n': len(values)
    }

stats_naive = calculate_stats(ua_orig_naive, "UA_orig (naive)")
stats_lobo = calculate_stats(ua_orig_lobo, "UA_orig (LOBO)")

# ==============================
# 5. 创建四线图表
# ==============================

print("\nCreating four-line figure...")

fig, ax = plt.subplots(figsize=(fig_width_inch, fig_height_inch))

# 颜色方案 - 使用Set2色系的四种颜色
colors = {
    'orig_naive': '#7C7C7C',        # 灰色: 原始乐观估计
    'orig_lobo': '#DD8452',         # 橙色: LOBO校正后的原始UA
    'adj': '#4C72B0',               # 蓝色: 仅基础率调整
    'adj_spatial': '#55A868'        # 绿色: 基础率+空间调整
}

line_styles = {
    'orig_naive': ':',        # 点线
    'orig_lobo': '--',        # 虚线
    'adj': '-',               # 实线
    'adj_spatial': '-.'       # 点划线
}

# ------------------------------
# 1. 绘制UA_original (naive) - 水平线
# ------------------------------
x_min = ua_adj_stats.index.min()
x_max = ua_adj_stats.index.max()
x_range = np.array([x_min, x_max])

line_orig_naive, = ax.plot(x_range, [stats_naive['mean'], stats_naive['mean']],
                          color=colors['orig_naive'], linewidth=2.0,
                          linestyle=line_styles['orig_naive'],
                          label='UA$_{orig}$ (Naive, balanced sampling)', zorder=3)

# ------------------------------
# 2. 绘制UA_original (LOBO) - 水平线
# ------------------------------
line_orig_lobo, = ax.plot(x_range, [stats_lobo['mean'], stats_lobo['mean']],
                         color=colors['orig_lobo'], linewidth=2.5,
                         linestyle=line_styles['orig_lobo'],
                         label='UA$_{orig}$ (Spatial LOBO-corrected)', zorder=4)

# ------------------------------
# 3. 绘制UA_adj (仅基础率调整)
# ------------------------------
line_adj, = ax.plot(ua_adj_stats.index, ua_adj_stats['mean'],
                   color=colors['adj'], linewidth=2.5,
                   marker='o', markersize=6, markeredgecolor='white', markeredgewidth=1,
                   label='UA$_{adj}$ (Base rate-adjusted only)', zorder=5)

# 添加UA_adj的25%-75%分位数范围阴影
ax.fill_between(ua_adj_stats.index,
                ua_adj_stats['q25'],
                ua_adj_stats['q75'],
                color=colors['adj'], alpha=0.2,
                label='UA$_{adj}$: 25%-75% range', zorder=1)

# ------------------------------
# 4. 绘制UA_adj (spatial LOBO adjusted)
# ------------------------------
line_adj_spatial, = ax.plot(ua_adj_spatial_stats.index, ua_adj_spatial_stats['mean'],
                           color=colors['adj_spatial'], linewidth=2.5,
                           marker='s', markersize=6, markeredgecolor='white', markeredgewidth=1,
                           label='UA$_{adj}$ (Base rate + Spatial LOBO)', zorder=6)

# 添加UA_adj_spatial的25%-75%分位数范围阴影
ax.fill_between(ua_adj_spatial_stats.index,
                ua_adj_spatial_stats['q25'],
                ua_adj_spatial_stats['q75'],
                color=colors['adj_spatial'], alpha=0.2,
                label='UA$_{adj}$ (spatial): 25%-75% range', zorder=2)

# ------------------------------
# 设置坐标轴属性
# ------------------------------
ax.set_xlabel('PFCL Base Rate (%)', fontsize=11, fontweight='bold')
ax.set_ylabel('User Accuracy (UA)', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)

ax.set_xlim(x_min - 0.5, x_max + 0.5)

# 设置y轴范围
y_min = max(0, min(
    ua_adj_stats['q25'].min(),
    ua_adj_spatial_stats['q25'].min(),
    stats_lobo['q25']
) - 0.05)

y_max = min(1.0, max(
    ua_adj_stats['q75'].max(),
    ua_adj_spatial_stats['q75'].max(),
    stats_lobo['q75']
) + 0.05)

ax.set_ylim(y_min, y_max)

# 设置刻度
y_ticks = np.arange(np.floor(y_min * 10) / 10, np.ceil(y_max * 10) / 10 + 0.05, 0.1)
ax.set_yticks(y_ticks)

base_rates = df['pfcl_base_rate_percent'].unique()
if len(base_rates) > 20:
    major_ticks = [0.5,  2, 5, 10, 15, 20]
    ax.set_xticks([x for x in major_ticks if x in base_rates])
else:
    ax.set_xticks(base_rates)

# ------------------------------
# 添加统计信息
# ------------------------------
n_ecoregions = df['ecoregion'].nunique()
mean_ua_adj = ua_adj_stats['mean'].mean()
mean_ua_adj_spatial = ua_adj_spatial_stats['mean'].mean()

# 计算各种偏差
bias_naive_vs_lobo = stats_naive['mean'] - stats_lobo['mean']
bias_adj_vs_adj_spatial = mean_ua_adj - mean_ua_adj_spatial
bias_adj_vs_lobo = stats_lobo['mean'] - mean_ua_adj
bias_adj_spatial_vs_lobo = stats_lobo['mean'] - mean_ua_adj_spatial






# ------------------------------


# ------------------------------
# 创建图例
# ------------------------------
# 收集所有线条
lines = [line_orig_naive, line_orig_lobo, line_adj, line_adj_spatial]
labels = [line_orig_naive.get_label(), line_orig_lobo.get_label(),
          line_adj.get_label(), line_adj_spatial.get_label()]

# 添加填充区域的图例句柄
from matplotlib.patches import Patch
patch_adj = Patch(facecolor=colors['adj'], alpha=0.2,
                  edgecolor=colors['adj'], linewidth=0.5,
                  label='UA$_{adj}$: 25%-75% range')
patch_adj_spatial = Patch(facecolor=colors['adj_spatial'], alpha=0.2,
                          edgecolor=colors['adj_spatial'], linewidth=0.5,
                          label='UA$_{adj}$(spatial): 25%-75% range')

lines.append(patch_adj)
lines.append(patch_adj_spatial)
labels.append(patch_adj.get_label())
labels.append(patch_adj_spatial.get_label())

# 创建图例（分两列，避免遮挡图表）
legend = ax.legend(lines, labels, loc='lower right',
                   fontsize=8.5, framealpha=0.9,
                   frameon=True, fancybox=True, ncol=2,
                   edgecolor='black', facecolor='white')
legend.get_frame().set_linewidth(0.5)

# ------------------------------
# 添加图表说明
# ------------------------------




plt.tight_layout()

# 保存图表
output_path = '/content/drive/MyDrive/fig_ua_four_lines_JAG.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none', format='png')
print(f"\nFigure saved to: {output_path}")
plt.show()

# ==============================
# 6. 输出详细统计对比
# ==============================

print("\n" + "="*80)
print("COMPREHENSIVE STATISTICAL COMPARISON OF FOUR METRICS")
print("="*80)

print(f"\n1. Summary Statistics:")
print(f"{'Metric':<40} {'Mean':>8} {'Std':>8} {'Min':>8} {'Max':>8}")
print("-" * 75)
print(f"{'UA_orig (Naive)':<40} {stats_naive['mean']:>8.3f} {stats_naive['std']:>8.3f} "
      f"{stats_naive['min']:>8.3f} {stats_naive['max']:>8.3f}")
print(f"{'UA_orig (LOBO)':<40} {stats_lobo['mean']:>8.3f} {stats_lobo['std']:>8.3f} "
      f"{stats_lobo['min']:>8.3f} {stats_lobo['max']:>8.3f}")
print(f"{'UA_adj (Base rate only)':<40} {mean_ua_adj:>8.3f} {df['UA_adj'].std():>8.3f} "
      f"{df['UA_adj'].min():>8.3f} {df['UA_adj'].max():>8.3f}")
print(f"{'UA_adj (Base rate + Spatial)':<40} {mean_ua_adj_spatial:>8.3f} {df['UA_adj_spatial'].std():>8.3f} "
      f"{df['UA_adj_spatial'].min():>8.3f} {df['UA_adj_spatial'].max():>8.3f}")

print(f"\n2. Bias Analysis (Absolute Differences):")
print(f"   A. Optimism bias (Naive vs LOBO): {bias_naive_vs_lobo:.4f}")
print(f"      Relative: {bias_naive_vs_lobo/stats_naive['mean']:.2%}")
print(f"   B. Base rate adjustment effect (UA_adj vs UA_orig naive): {stats_naive['mean'] - mean_ua_adj:.4f}")
print(f"      Relative: {(stats_naive['mean'] - mean_ua_adj)/stats_naive['mean']:.2%}")
print(f"   C. Spatial adjustment effect on UA_adj: {bias_adj_vs_adj_spatial:.4f}")
print(f"      Relative: {bias_adj_vs_adj_spatial/mean_ua_adj:.2%}")
print(f"   D. Total adjustment (UA_adj_spatial vs UA_orig naive): {stats_naive['mean'] - mean_ua_adj_spatial:.4f}")
print(f"      Relative: {(stats_naive['mean'] - mean_ua_adj_spatial)/stats_naive['mean']:.2%}")

print(f"\n3. Comparison with LOBO-corrected baseline:")
print(f"   A. UA_adj vs UA_orig (LOBO): {bias_adj_vs_lobo:+.4f} "
      f"({'lower' if bias_adj_vs_lobo > 0 else 'higher'})")
print(f"   B. UA_adj_spatial vs UA_orig (LOBO): {bias_adj_spatial_vs_lobo:+.4f} "
      f"({'lower' if bias_adj_spatial_vs_lobo > 0 else 'higher'})")
print(f"   C. Which is closer to LOBO baseline? "
      f"{'UA_adj_spatial' if abs(bias_adj_spatial_vs_lobo) < abs(bias_adj_vs_lobo) else 'UA_adj'}")

print(f"\n4. Trend Analysis:")
if len(ua_adj_stats) > 1:
    print(f"   UA_adj slope: {slope_adj:.6f} per 1% base rate increase")
    print(f"     Correlation (r): {r_adj:.4f}, p-value: {p_adj:.4e}")
    print(f"   UA_adj_spatial slope: {slope_spatial:.6f} per 1% base rate increase")
    print(f"     Correlation (r): {r_spatial:.4f}, p-value: {p_spatial:.4e}")

    # 比较斜率差异
    slope_diff = slope_adj - slope_spatial
    print(f"   Slope difference: {slope_diff:.6f}")
    print(f"   Interpretation: Spatial adjustment {'increases' if slope_diff > 0 else 'decreases'} "
          f"the sensitivity to base rate by {abs(slope_diff):.6f} per 1%")

print(f"\n5. Key Insights:")
print(f"   • Total adjustment reduces UA by {(stats_naive['mean'] - mean_ua_adj_spatial)/stats_naive['mean']*100:.1f}% on average")
print(f"   • Spatial LOBO correction accounts for {bias_naive_vs_lobo/(stats_naive['mean'] - mean_ua_adj_spatial)*100:.1f}% of total adjustment")
print(f"   • Base rate adjustment accounts for {(stats_naive['mean'] - mean_ua_adj)/(stats_naive['mean'] - mean_ua_adj_spatial)*100:.1f}% of total adjustment")
print(f"   • Combined correction provides the most realistic estimate")

print(f"\n6. Output File:")
print(f"   {output_path}")
print("="*80)

# 保存调整后的数据以供进一步分析
output_data_path = '/content/drive/MyDrive/ua_adjusted_data_with_spatial.csv'
df[['ecoregion', 'pfcl_base_rate_percent', 'UA_adj', 'UA_adj_spatial']].to_csv(output_data_path, index=False)
print(f"\nAdjusted data saved to: {output_data_path}")